In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
%reload_kedro

In [ ]:
from crypto_thesis.utils import optimize_params
from pprint import pprint
from sklearn.linear_model import LogisticRegression
import numpy as np
from time import time

## Base

In [ ]:
TARGET_COL = ["label"]
INDEX_COL = "window_nbr"

N_SPLITS = 5

In [ ]:
mt_train_nonmultic = catalog.load("master_table_train_nonmultic")

## LogReg

In [ ]:
logreg_def_params = catalog.load("params:logreg_default_params")

master_table_train = mt_train_nonmultic.set_index(INDEX_COL).sort_index().reset_index(drop=True)
X_train, y_train = master_table_train.drop(columns=TARGET_COL), master_table_train[TARGET_COL]

model = LogisticRegression(**logreg_def_params)

In [ ]:
# elastic net
space = {
        "solver": ["saga"],
        "penalty": ["elasticnet"],
        "tol": np.logspace(-4, 1, 5),
        "C": np.logspace(-2, 1, 5),
        "max_iter": list(range(100, 1100, 200)),
        "fit_intercept": [True, False],
        "class_weight": ["balanced", None],
        "l1_ratio": np.logspace(-2, 0, 5),
        "random_state": [0]
    }

start = time()
params_opt = optimize_params(model=model,
                            grid=space,
                            X_train=X_train,
                            y_train=y_train,
                            n_splits=N_SPLITS)
end = time()

print("*" * 100)
print()
print("Best parameters:")
pprint(params_opt.best_params_)
print()
print(f"Best score: {params_opt.best_score_}")
print()
print(f"Time elapsed (seconds): {end-start}")

In [ ]:
# l1
space2 = {
        "solver": ["saga", "liblinear"],
        "penalty": ["l1"],
        "tol": np.logspace(-4, 1, 5),
        "C": np.logspace(-2, 1, 5),
        "max_iter": list(range(100, 1100, 200)),
        "fit_intercept": [True, False],
        "class_weight": ["balanced", None],
        "random_state": [0]
    }

start = time()
params_opt = optimize_params(model=model,
                            grid=space2,
                            X_train=X_train,
                            y_train=y_train,
                            n_splits=N_SPLITS)
end = time()

print("*" * 100)
print()
print("Best parameters:")
pprint(params_opt.best_params_)
print()
print(f"Best score: {params_opt.best_score_}")
print()
print(f"Time elapsed (seconds): {end-start}")

In [ ]:
# l2
space3 = {
        "solver": ["saga", "lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag"],
        "penalty": ["l2"],
        "tol": np.logspace(-4, 1, 5),
        "C": np.logspace(-2, 1, 5),
        "max_iter": list(range(100, 1100, 200)),
        "fit_intercept": [True, False],
        "class_weight": ["balanced", None],
        "random_state": [0]
    }

start = time()
params_opt = optimize_params(model=model,
                            grid=space3,
                            X_train=X_train,
                            y_train=y_train,
                            n_splits=N_SPLITS)
end = time()

print("*" * 100)
print()
print("Best parameters:")
pprint(params_opt.best_params_)
print()
print(f"Best score: {params_opt.best_score_}")
print()
print(f"Time elapsed (seconds): {end-start}")